# Машинное обучение ранжированию с помощью библиотеки LightGBM

В этом примере мы:

- познакомимся с библиотекой **LightGBM**
- научимся решать задачу машинного обучения ранжирования используя алгоритмы, реализованные в **LightGBM**

Будем использовать стандартный датасет [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)

Подробности про формат датасета можно найти в соседнем тюториале *catboost_ltr.ipynb* по машинному обучению ранжированию с помощью библиотеки **CatBoost**, предполагаем что мы его уже прошли.

## Библиотека LightGBM

Полезные ссылки:
- Полная документация: https://lightgbm.readthedocs.io/en/stable/
- класс LGBMRanker: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRanker.html

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [1]:
from timeit import default_timer as timer

import pandas as pd

import lightgbm as lgbm
# We'll use catboost to download dataset and to compute metrics
from catboost import datasets, utils

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим встроенный в катбуст сабсет датасета MSLR:

In [2]:
df_train, df_test = datasets.msrank_10k()

Для удобства присвоим колонкам говорящие имена:

In [3]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns

# Assign column names
columns = generate_column_names(num_features=136)
df_train.columns = columns
df_test.columns = columns

Теперь наши данные выглядят красивее:

In [4]:
print(df_train.head(5))

   label  qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0    1          3          3          0          0          3   
1    2.0    1          3          0          3          0          3   
2    0.0    1          3          0          2          0          3   
3    2.0    1          3          0          3          0          3   
4    1.0    1          3          0          3          0          3   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        1.0   0.000000  ...           62     11089534   
1        1.0        0.0   1.000000  ...           54     11089534   
2        1.0        0.0   0.666667  ...           45            3   
3        1.0        0.0   1.000000  ...           56     11089534   
4        1.0        0.0   1.000000  ...           64            5   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0            2          116        64034           13            3   
1           

Всего у нас:

- 87 запросов и 10000 документов в трейне
- 88 запросов и 10000 документов в тесте
- 130 колонок: таргет (оценка асессора), id запроса (qid) и вектор из 128 фичей
- таргет принимает значения в интервале \[0,4\]

Теперь нам надо представить датасет в формате, который можно подавать на вход модели lightgbm.

В отличие от catboost и xgboost, которые "связывают" документы в рамках одного запроса по ID запроса, lightgbm кодирует группы немного по-другому.

Мы все еще делим датасет на 3 части, только вместо вектора **q** у нас теперь вектор **g**:

- **y** -- вектор таргетов
- **X** -- тензор из фичей
- **g** -- вектор c размерами групп

Мы предполагаем, что документы в **y** и **X** сгруппированы и отсортированы по qid, а вот **g** теперь просто содержит размеры соответсвующих групп, и его размер равен не количеству объектов, а количеству групп.

Проще всего понять это на примере.

Будем использовать тот факт, что наш датасет уже отсортирован по qid.

Убедимся в этом:

In [5]:
print(df_train['qid'].is_monotonic_increasing)
print(df_test['qid'].is_monotonic_increasing)

True
True


Сконвертируем датасет в нужный формат:

In [6]:
def to_lightgbm_dataset(df):
    y = df['label'].to_numpy()                       # Label: [0-4]
    q = df['qid'].to_numpy().astype('uint32')        # Query Id
    X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features

    num_docs_by_qid = df.groupby('qid')['label'].count()
    g = num_docs_by_qid.to_numpy()
    return (X, y, q, g)

X_train, y_train, q_train, g_train = to_lightgbm_dataset(df_train)
X_test, y_test, q_test, g_test = to_lightgbm_dataset(df_test)

Посмотрим как выглядят вектора **q** и **g**:

In [7]:
print("Targets and queries:")
print(pd.DataFrame.from_dict({'y': y_train, 'q': q_train}).head(5))

print("\nGroups:")
print(pd.DataFrame.from_dict({'g': g_train}).head(5))

Targets and queries:
     y  q
0  2.0  1
1  2.0  1
2  0.0  1
3  2.0  1
4  1.0  1

Groups:
     g
0   86
1  106
2   92
3  120
4   59


Вектор **q** нам также понадобится в дальнейшем для расчета метрик, поэтому to_lightgbm_dataset() возвращает сразу и его.

## Обучаем pointwise модель

Теперь можно приступить непосредственно к обучению модели. Мы начнем с простой pointwise модели которая в качестве лосса использует обычное RMSE.

Количество деревьев будем выбирать автоматически с использованием т.н. early stopping -- процесс обучения будем остановлен после того, как на валидационном множестве перестанет расти наша целевая метрика (в нашем случае это тоже RMSE).

Создадим объект модели:

In [8]:
model = lgbm.LGBMRegressor(
    n_estimators=1000, # maximum possible number of trees
    random_state=22
)

И зафитим ее на нашем обучающем множестве.  
Т.к. мы используем early stopping то передадим в функцию fit() в качестве валидационного множества (параметр eval_set) наш тест-сет (строго говоря так делать нельзя, но мы в данном примере не стремимся к строгости, подробнее этот момент расписан в тюториале про машинное обучение ранжированию с помощью каибуста).

In [9]:
# Fit
start = timer()
model.fit(X_train, y_train,
    eval_set=[(X_test, y_test)],
    callbacks=[lgbm.early_stopping(stopping_rounds=100, first_metric_only=True)],
)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002697 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20855
[LightGBM] [Info] Number of data points in the train set: 10000, number of used features: 136
[LightGBM] [Info] Start training from score 0.628200
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[35]	valid_0's l2: 0.543193
Evaluated only: l2
Model fit: elapsed = 0.509


Посмотрим, сколько деревьев содержит модель. Это немного сложнее чем было с катбустом:

In [10]:
num_trees = model.booster_.num_trees()
print(f"Model params: num_trees = {num_trees} best_iteration = {model.best_iteration_}")

Model params: num_trees = 35 best_iteration = 35


Видно, что наша модель содержит 35 деревьев (и все 35 будут использованы для предикта).

При желании, мы можем сохранить модель в текстовом формате:

In [11]:
model_file = "/tmp/model.lgbm.txt"

# Save model
model.booster_.save_model(model_file)

# Load model
# model = lgbm.Booster(model_file=model_file)

Получим предикты модели на тестовом множестве:

In [12]:
y_hat_test = model.predict(X_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Predicted: y_hat_test.shape = (10000,)


Теперь, имея предикты, можно посчитать метрики качества, тут для удобства и единообразия будем использовать catboost:

In [13]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp', 'DCG:top=10;type=Exp', 'MAP:top=10']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

metric = NDCG:top=10;type=Exp score = 0.408
metric = DCG:top=10;type=Exp score = 8.236
metric = MAP:top=10 score = 0.520


Видим, что значение NDCG на тесте получилось чуть хуже, чем у катбуста.

## Обучаем pairwise модели

Теперь проделаем все то же самое, но с использованием алгоритма LambdaRank.  

Для этого вместо *LGBMRegressor* будем использовать класс *LGBMRanker*, которому в качестве objective передадим 'lambdarank'.  
В качестве метрики для early stopping на этот раз будем использовать ndcg.

Функция fit() теперь принимает на вход еще и информацию о группах.

In [14]:
# Create model
model = lgbm.LGBMRanker(
    n_estimators=1000,         # maximum possible number of trees
    objective='lambdarank',    # LambdaRank objective
    metric='ndcg',             # metric used for early stopping
    random_state=22
)

# Fit
start = timer()
model.fit(X_train, y_train, group=g_train,
    eval_set=[(X_test, y_test)], eval_group=[g_test], eval_at=[10],
    callbacks=[lgbm.early_stopping(stopping_rounds=10, first_metric_only=True)],
)
elapsed = timer() - start
print(f"Model fit: num_trees = {model.best_iteration_} elapsed = {elapsed:.3f}")

# Predict
y_hat_test = model.predict(X_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

# Compute metrics on test
print("\nEvaluated:")
compute_metrics(y_test, y_hat_test, q_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005537 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20855
[LightGBM] [Info] Number of data points in the train set: 10000, number of used features: 136
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[27]	valid_0's ndcg@10: 0.409668
Evaluated only: ndcg@10
Model fit: num_trees = 27 elapsed = 0.283
Predicted: y_hat_test.shape = (10000,)

Evaluated:
metric = NDCG:top=10;type=Exp score = 0.409
metric = DCG:top=10;type=Exp score = 8.253
metric = MAP:top=10 score = 0.524


Сравним результаты:

- RMSE модель выбила NDCG@10 = **0.408**
- а LambdaRank выбивает NDCG@10 = **0.409**

Т.е. в данном случае мы не видим "из коробки" преимущества pairwise-подхода над poinwise, во всяком случае без дополнительного тюнинга гиперпараметров. 

Также, обратим внимание, что абсолютные значения NDCG@10 даже у LambdaRank хуже того, что выбивала простая регрессия в catboost (**0.419**), не говоря уже о YetiRank!

Видно, что "из коробки" catboost показывает значительно лучшие результаты, чем lightgbm.  